생육 환경 최적화 경진대회
https://dacon.io/competitions/official/235897/overview/description

4월 18일 오픈
 train [Folder] : 학습용 데이터셋 (1009장)

	│	├ CASE01 : 각 케이스 별 데이터

	│	│	├ image : 이미지 데이터

	│	│	├ meta : 환경 데이터

	│	│		└ 촬영 된 시각으로부터 1일동안 측정된 '내부온도', '내부습도', 'CO2', 'EC' 의 환경정보

	│	│	└ label.csv :

	│	│		└ img_name : 해당 이미지 파일명

	│	│		└ leaf_weight : 해당 이미지로부터 1일 후의 잎 면적 (중량)

	│	├ CASE02

	│	├ CASE03

	│	└ ...

 test [Folder] : 테스트용 데이터셋 (342장)

	│	├ image : 이미지 데이터

	│	├ meta : 환경 데이터

	│		└ 촬영 된 시각으로부터 1일동안 측정된 '내부온도', '내부습도', 'CO2', 'EC' 의 환경정보

	└ sample_submission.csv

		└ img_name : 해당 이미지 파일명

		└ leaf_weight : 해당 이미지로부터 1일 후의 잎 면적 (중량) 예측 값

In [21]:
import pandas as pd
import numpy as np

In [22]:
data = pd.read_csv('./dataset/train/CASE01/meta/CASE01_01.csv')

In [23]:
np.shape(data)

(1440, 19)

In [24]:
data.head()

,시간,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,냉방온도,냉방부하,난방온도,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,2021-05-07 02:46,18.6,41.500000,77.599998,NaN,582.0,18.600000,0.0,201.0,201.0,201.0,14.0,0.0,15.0,13.000002,0.0,0.0,0.0,NaN
1,2021-05-07 02:47,18.6,41.500000,77.699997,0.0,585.0,18.799999,0.0,201.0,201.0,0.0,14.0,0.0,15.0,13.000002,0.0,0.0,0.0,NaN
2,2021-05-07 02:48,18.6,41.299999,77.800003,0.0,587.0,18.700001,0.0,201.0,201.0,NaN,14.0,0.0,15.0,13.000002,0.0,0.0,0.0,NaN
3,2021-05-07 02:49,18.5,41.099998,77.800003,0.0,584.0,18.799999,0.0,201.0,201.0,0.0,14.0,0.0,15.0,12.500000,0.0,0.0,0.0,NaN
4,2021-05-07 02:50,18.6,41.200001,77.800003,0.0,583.0,19.100000,0.0,201.0,201.0,0.0,14.0,0.0,15.0,13.000002,0.0,0.0,0.0,NaN


In [25]:
data['시간']

0       2021-05-07 02:46
1       2021-05-07 02:47
2       2021-05-07 02:48
3       2021-05-07 02:49
4       2021-05-07 02:50
              ...       
1435    2021-05-08 02:41
1436    2021-05-08 02:42
1437    2021-05-08 02:43
1438    2021-05-08 02:44
1439    2021-05-08 02:45
Name: 시간, Length: 1440, dtype: object

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시간           1440 non-null   object 
 1   내부온도관측치      1440 non-null   float64
 2   외부온도관측치      1440 non-null   float64
 3   내부습도관측치      1440 non-null   float64
 4   외부습도관측치      1149 non-null   float64
 5   CO2관측치       1440 non-null   float64
 6   EC관측치        1440 non-null   float64
 7   최근분무량        1440 non-null   float64
 8   화이트 LED동작강도  1387 non-null   float64
 9   레드 LED동작강도   1389 non-null   float64
 10  블루 LED동작강도   860 non-null    float64
 11  냉방온도         1416 non-null   float64
 12  냉방부하         1416 non-null   float64
 13  난방온도         1416 non-null   float64
 14  난방부하         1416 non-null   float64
 15  총추정광량        1416 non-null   float64
 16  백색광추정광량      1416 non-null   float64
 17  적색광추정광량      1416 non-null   float64
 18  청색광추정광량      0 non-null      float64
dtypes: flo

### 레이어를 구성하고 검증

In [7]:
import numpy as np
import random
import os
import math

from glob import glob
# https://wikidocs.net/83
import pandas as pd
import cv2
from tqdm.auto import tqdm
# https://frhyme.github.io/python-libs/python_tqdm/

import torch
import torch.nn as nn # 신경망 호출
import torch.nn.functional as F #nn과 똑같지만 nn은 클래스로 정의되고 functional은 함수로 정의됨
from torch.utils.data import DataLoader, Dataset
#데이터 로딩을 위한 클래스 https://huffon.github.io/2020/05/26/torch-data/

import torchvision.models as models
from torchvision import transforms
#https://better-tomorrow.tistory.com/entry/TorchVision-model-funetuning

from torchsummary import summary

### Custom 레이어 구성

In [36]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(
            #[8,3,256,256] -> [8,8,256,256]
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(),
            #[8,256,256] -> [8,128,128]
            nn.MaxPool2d(kernel_size=2, stride=2))
            
            #[8,8,256,256] -> [8,16,128,128]
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            #[16,128,128]-> [16,64,64]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,16,64,64] -> [8,32,64,64]
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            #[8,32,64,64]-> [8,32,32,32]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,32,32,32] -> [8,64,32,32]
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            #[8,64,32,32] -> [8,64,16,16]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,64,16,16] -> [8,128,14,14]
        self.layer5 = torch.nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            #[8,128,14,14] -> [8,128,7,7]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.regressor=nn.Linear(128*7*7,1)
        # 보통 여기서는 fc, full conneted network으로 불리는 선형출력
        # nn.Linear()는 입력의 차원, 출력의 차원 레이어를 만들어 본다.
        # 이런 레이어 대신 VGG나 ResNet으로하면 성능 향상이 더 있을까?

    def forward(self, x):
        # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
        # (Batch, 1, 256, 256)
        x = self.layer1(x)
        # (Batch, 8, 128, 128)
        x = self.layer2(x)
        # (Batch, 16, 64, 64)
        x = self.layer3(x)
        # (Batch, 32, 32, 32)
        x = self.layer4(x)
        # (Batch, 64, 16, 16)
        x = self.layer5(x)
        # (Batch, 128, 7, 7) -> Flatten (Batch, 128*7*7)
        x = torch.flatten(x, start_dim=1)
        # Regressor (Batch, 128*7*7) -> (Batch, 1)
        out = self.regressor(x)
        return out

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# 쿠다를 사용할수 있으면 쿠다를 사용하고 아니면 CPU로 연산

In [38]:
model = CNNRegressor().to(device)

### summary 라이브러리를 이용하여 레이어검증

In [39]:
summary(model,input_size=(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 256, 256]             224
       BatchNorm2d-2          [-1, 8, 256, 256]              16
         LeakyReLU-3          [-1, 8, 256, 256]               0
         MaxPool2d-4          [-1, 8, 128, 128]               0
            Conv2d-5         [-1, 16, 128, 128]           1,168
       BatchNorm2d-6         [-1, 16, 128, 128]              32
         LeakyReLU-7         [-1, 16, 128, 128]               0
         MaxPool2d-8           [-1, 16, 64, 64]               0
            Conv2d-9           [-1, 32, 64, 64]           4,640
      BatchNorm2d-10           [-1, 32, 64, 64]              64
        LeakyReLU-11           [-1, 32, 64, 64]               0
        MaxPool2d-12           [-1, 32, 32, 32]               0
           Conv2d-13           [-1, 64, 31, 31]          32,832
      BatchNorm2d-14           [-1, 64,

In [11]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(64, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer5 = torch.nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=4,stride =1, padding=1),
            nn.BatchNorm2d(64,momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.regressor = nn.Linear(64*3*3, 1)
        # nn.Linear()는 입력의 차원, 출력의 차원

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = torch.flatten(x, start_dim=1)
        out = self.regressor(x)
        
        return out

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = CNNRegressor().to(device)
summary(model,input_size=(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 128, 128]             224
       BatchNorm2d-2          [-1, 8, 128, 128]              16
         LeakyReLU-3          [-1, 8, 128, 128]               0
         MaxPool2d-4            [-1, 8, 64, 64]               0
            Conv2d-5           [-1, 16, 64, 64]           1,168
       BatchNorm2d-6           [-1, 16, 64, 64]              32
         LeakyReLU-7           [-1, 16, 64, 64]               0
         MaxPool2d-8           [-1, 16, 32, 32]               0
            Conv2d-9           [-1, 32, 32, 32]           4,640
      BatchNorm2d-10           [-1, 32, 32, 32]              64
        LeakyReLU-11           [-1, 32, 32, 32]               0
        MaxPool2d-12           [-1, 32, 16, 16]               0
           Conv2d-13           [-1, 64, 15, 15]          32,832
      BatchNorm2d-14           [-1, 64,